# Markov Chain Monty Carlo Simulations
Describe MCMC

# The Detailed Balance Equation
The detailed balance equation (DBE) is a condition used to converge a MCMC simulation to a distribution -- and in this case the thermodynamic Boltzmann distribution. The DBE is a statement that the transition probabilty from one state $\xi$ to some new state $\xi'$ has an equal probability, and can be stated as such:

$$P(\xi\rightarrow\xi') = P(\xi'\rightarrow\xi)$$

The Boltzmann distribution is a distribution that gives a probability that a system will be in a state given the macroenergy of that states collective microstates, and is given by:

$$p(\xi)\propto e^{-E_i\beta}$$

Using this as a cost function of our systems energy and the DBE, we can write:

$$p(\xi) = p(\xi')$$
$$\implies e^{-E_i\beta} = e^{-E_f\beta}$$

Taking the ratio of this, we arrive at our cost function for a MCMC evolution to accept or reject a proposed change in the system:

$$\frac{p(\xi)}{p(\xi')} = e^{-\beta(\Delta E)}$$

Therefore for a given choice of $\beta J$, the only deciding factor for accepting or rejecting an evolution is the change in energy of the system from $\xi$ to $\xi'$. We can use the previously derived energy $-\sum_{<i,j>} J_{<i,j>} \sigma_i \sigma_j$, which is the sum neighboring spins times the spin connecting all neighbors.

Using this fact, we can compute the total energy of the system once, then only compute the sum of neighboring spins of a local microstate and multiply this by two times the sign of the proposed spin flip to get the change in energy from $\xi$ to $\xi'$. Using this fact and calling the resulting sum $\Sigma_{nbr}$ and the flip of the initially chosen spin $S_f$, we get:

$$\implies \frac{p(\xi)}{p(\xi')} = e^{-2S_f\Sigma_{nbr}\beta J}$$

# The dyanmics of spin flips

talk about why phonons only travel from the boundarys of spin configurations

# The Metropolis Algorithm
The Metropolis algorithm is a MCMC method that utlizes the cost function to flip a single randomly (or pseudo randomly) selected spin per itteration. The goal of the algorithm is to nudge the system from a state $\Xi$ into a lower energy state $\Theta$.

The main drawback of this algorithm comes from its treatment of spin flips, and warrents a look into the dyanmics of an interacting spin model.

The wavefunction that describes each individual spin in the lattice has no boundary on its effective influence range, however for simplicity we only consider nearest neighbors. The problem with this simplification is that near the critical transition point, single flip interactions based only on lowering the total energy of the system do not addequatly transmit spin-spin relaxation signals, as the next selected spin is not garunteed to be near the spin flipped in the current itteration.

# The Wolff Algorithm
talk about the treatment of long wavelength phonon transmition in the wolf model and how it overcomes the critical slowdown near the critical temperature
talk about how the DBE can be changed to be $e^{-2\beta J}$ due to the change in energy always being a factor of $\pm 2$ due to only selecting clusters.

# Plotting relations
We can look at $\bar{m}$ as a function of the temperature by using the thermodynamic relationship of $\beta = \frac{1}{k_B T}$. Next, we can expand the partition function as so:

$$ Z = \sum_{<i,j>} e^{-\beta E_i} $$
$$ E = -\sum_{<i,j>} J_{<i,j>} \sigma_i \sigma_j,\quad \sigma_i \sigma_j \in \mathbb{Z}$$
$$ \implies Z = a e^{-\beta J},\quad a\in\mathbb{R} $$

From this we can see that the available microstates depend on the $\beta J$ value, and are scaled with the specific spin arrangemet, where $\beta J$ is a dimensionless quantity useful for numerical computation.

## The Heat Capacity and Energy of the system

Using the partition function, we can also plot the heat capacity:

$$\langle E \rangle = -\frac{\partial ln(Z)}{\partial \beta}$$

$$\langle E^2 \rangle = \frac{\partial^2 ln(Z)}{\partial \beta^2}$$

With these two values, we can numerically compute  the heat capacity as:

finish derivation

$$C_V = \frac{\sigma_E^2}{k_b T^2} $$

From this result, we see that we only need to measure the energy at each evolution step in a MCMC simulation, take the numerical standard deviation, and record the temperature $T$ used in each itteration of the simulation to plot the Heat Capacity as a function of a given $\beta J$.

## Magnitization density

Then, we use the above relations to first plot $\bar{m}$ vs $\beta J$ -- where $\bar{m}$ is the average of sum spins per evolution.

In [1]:
"""
This file is the entry point for the project
"""
# Begin by importing external libraries using path_setup
# NOTE : Must be ran first, thus PEP8-E402
import path_setup
path_setup.path_setup()
import math  # noqa E402
import sys  # noqa E402
import numpy as np # noqa E402
import datetime as dt # noqa E402
from LatticeDriver import LatticeDriver as lt  # noqs E402
from LatticeDriver import T_to_Beta  # noqa E402
import random as rnd  # noqa E402
from SupportingFunctions import generate_random, rand_time, DividendRemainder
# Shebang line for interactive output in vs_code, comment this out if you have troubles running the notebook
import plotly.io as pio
pio.renderers.default = "notebook"

zeroC = 273.15

Linux System



# Setup Parameters
This is the setup for the simulation.

N, M
> The span of the basis vectors

$\beta$ and $J$
> `Beta` represents a range of values for the thermodynamic variable in the partition function, which si equal to $\frac{1}{k_bT}$.

> `J` represents the spin coupling value to use lattice wide. Later I want to provide a base value and have the distance between spin sites use a different J value. Useful for phonons that propogate and lattices that are not using C4V symmetry.

Options 0 and 1
> 0 for seeded random or 1 for time based
Seeded random : 0
> seeds random with 1644121893 by default to generate a repeatable test.

```Probs``` is a list containing 2 -- or 3 integers if voids is enabled, from 0 to 100 such that they all add together to 100.
Each entry in the ```Probs``` list represents the percent chance that random will assign a spin value of 1, -1, or
0 if lattice voids are enabled. You can play with these for interesting results. 
Time based : 1
> seeds radom with the current epoch time as an integer.

In [2]:
N = 64
M = 64
size = [N, M]
total_time = 1000
J = 0.01  # eV, interation energy
T1 = zeroC
T2 = 1000+zeroC
a = T_to_Beta(T1)
b = T_to_Beta(T2)
print(f'a={a}/eV, b={b}/eV')
num_points = 100
step = (T2-T1)/num_points
Beta = np.zeros(num_points, dtype=np.float64)
for i in range(num_points):
    Beta[i] = T_to_Beta(i*step + T1)

a=42.48404950300416/eV, b=9.114808248631807/eV


In [3]:
lt_c4v_up = lt(1, size, J)
lt_c3v_up = lt(1, size, J, basis=[[1, 0], [0.5, np.sqrt(3)/2]])
lt_c6v_up = lt(1, size, J, [[0.5, np.sqrt(3)/2], [0.5, -np.sqrt(3)/2]])

lt_c4v_dn = lt(1, size, J)
lt_c3v_dn = lt(1, size, J, [[1, 0], [0.5, np.sqrt(3)/2]])
lt_c6v_dn = lt(1, size, J, [[0.5, np.sqrt(3)/2], [0.5, -np.sqrt(3)/2]])

output = input('Enter 0 for seeded random or 1 for time based:')
if output == '0':
    print("option 0 chosen.\n")
    # DOCtest seed = 1644121893
    seed = 1644121893
    lt_c4v_up.randomize(voids=True, probs=[20, 75, 5],
                        rand_seed=seed)
    lt_c3v_up.randomize(voids=False, probs=[25, 75],
                        rand_seed=seed)
    lt_c6v_up.randomize(voids=True, probs=[20, 75, 5],
                        rand_seed=seed)
    lt_c4v_dn.randomize(voids=True, probs=[75, 20, 5],
                        rand_seed=seed)
    lt_c3v_dn.randomize(voids=False, probs=[75, 25],
                        rand_seed=seed)
    lt_c6v_dn.randomize(voids=True, probs=[75, 20, 5],
                        rand_seed=seed)
else:
    print("option 1 chosen.\n")
    output = input('Enable voids (y/n)?')
    voids_enable = True if output == 'y' else False
    rand_n = 2 if voids_enable is False else 3
    seed = rand_time()

    lt_c4v_up.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c3v_up.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c6v_up.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c4v_dn.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c3v_dn.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)
    lt_c6v_dn.randomize(voids=voids_enable,
                        probs=generate_random(rand_n),
                        rand_seed=seed)

Generation complete!                                                                                                              
Generation complete!                                                                                                              
Generation complete!                                                                                                              
Generation complete!                                                                                                              
Generation complete!                                                                                                              
Generation complete!                                                                                                              


# Display
Calling <lattice_object>.plot() will display the current spin arangement of the lattice_object.

In [ ]:
lt_c4v_up.plot()
lt_c3v_up.plot()
lt_c6v_up.plot()
lt_c4v_dn.plot()
lt_c3v_dn.plot()
lt_c6v_dn.plot()

# Spin Energy and Magnitization
The next function up is `SpinEnergy`, which can be passed the positional parameter `lt.WolffAlgorithm` or `lt.MetropolisAlgorithm` deppending on which evolution method you want to test for a given range of $\beta$ values.

In [ ]:
lt_c4v_up.SpinEnergy(Beta, total_time, lt.WolffAlgorithm,
                     save=False, auto_plot=True);

In [ ]:
lt_c3v_up.SpinEnergy(Beta, total_time, lt.WolffAlgorithm,
                     save=False, auto_plot=True);

In [ ]:
lt_c6v_up.SpinEnergy(Beta, total_time, lt.WolffAlgorithm,
                     save=False, auto_plot=True);

In [ ]:
lt_c4v_dn.SpinEnergy(Beta, total_time, lt.WolffAlgorithm,
                     save=False, auto_plot=True);

In [ ]:
lt_c3v_dn.SpinEnergy(Beta, total_time, lt.WolffAlgorithm,
                     save=False, auto_plot=True);

In [ ]:
lt_c6v_dn.SpinEnergy(Beta, total_time, lt.WolffAlgorithm,
                     save=False, auto_plot=True);